In [19]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

from config import api_key

import numpy as np
import requests
import json

In [7]:
# Import cities file as DataFrame
twelve_thirteen_pd = pd.read_csv("draft_data/2012-13.csv")
twelve_thirteen_pd.head()

,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t CO2-e),Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e),Important Notes
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"595,404",30,297,327,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,405,976",94,"1,095","1,189",NaN
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,001,380",124,"1,392","1,516",NaN
3,AGL ENERGY LIMITED,AGL Loy Yang Power Station and Mine,F,"58,566,999","18,868,250","259,065","19,127,315",NaN
4,AGL ENERGY LIMITED,Banimboola Hydro,F,"41,286",3,69,72,NaN


In [8]:
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
twelve_thirteen_pd["Lat"] = ""
twelve_thirteen_pd["Lng"] = ""
twelve_thirteen_pd["Place_ID"] = ""
twelve_thirteen_pd["Address"] = ""
twelve_thirteen_pd.head()

,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t CO2-e),Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e),Important Notes,Lat,Lng,Place_ID,Address
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"595,404",30,297,327,NaN,,,,
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,405,976",94,"1,095","1,189",NaN,,,,
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,001,380",124,"1,392","1,516",NaN,,,,
3,AGL ENERGY LIMITED,AGL Loy Yang Power Station and Mine,F,"58,566,999","18,868,250","259,065","19,127,315",NaN,,,,
4,AGL ENERGY LIMITED,Banimboola Hydro,F,"41,286",3,69,72,NaN,,,,


In [14]:
twelve_thirteen_pd = twelve_thirteen_pd.drop(columns=["Important Notes"])
twelve_thirteen_pd.head()

,Reporting Entity,Facility Name,Type,Electricity Production (GJ),Greenhouse Gas Emissions Scope 1 (t CO2-e),Greenhouse Gas Emissions Scope 2 (t CO2-e),Total Emissions (t CO2-e),Lat,Lng,Place_ID,Address
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,F,"595,404",30,297,327,,,,
1,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,F,"2,405,976",94,"1,095","1,189",,,,
2,ACCIONA ENERGY OCEANIA PTY LTD,Corporate Total,C,"3,001,380",124,"1,392","1,516",,,,
3,AGL ENERGY LIMITED,AGL Loy Yang Power Station and Mine,F,"58,566,999","18,868,250","259,065","19,127,315",,,,
4,AGL ENERGY LIMITED,Banimboola Hydro,F,"41,286",3,69,72,,,,


In [22]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twelve_thirteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']

    # update address key value
    params['address'] = f"{location}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()

    twelve_thirteen_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    twelve_thirteen_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
twelve_thirteen_pd.head()

IndexError: list index out of range

In [2]:
engine = create_engine("sqlite:///draft_data/2012-13.sqlite")

In [3]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [4]:
# We can view all of the classes that automap found
Base.classes.keys()

[]